In [1]:
import torch, pickle, os
import torch_geometric as tg
from torch_geometric.data import Data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from torch_geometric.data import Dataset, Data
import os.path as osp
 
path='../../../../tigress/mcranmer/merger_trees/isotrees/'
print(f'Importing data ')
pd1=pd.read_table(path+'isotree_0_0_0.dat', skiprows=0, delimiter='\s+') # may want to just import target, can't enforce dtype
raw=pd1.drop(axis=0, index=np.arange(50)).reset_index()
trees=raw[raw.isna()['A[z]']] 
halos=raw[~raw.isna()['A[z]']] 



Importing data 


/home/cj1223/.conda/envs/juptorch/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
##### I might be able to shorten this a bit #####
f,i="float64", "int64"
cs=[f,i,f,i,i,i,i,i,i,f,f,f,f,f,i,f,f,f,f,f,f,f,f,f,f,f]
dicts = {}
keys = halos.columns[1:26]
castto = cs
for i, key in enumerate(keys):
        dicts[key] = castto[i]

halos=halos.astype(dicts) #### missing normalization of features

###### End ####
print('Splitting data')
spli=np.split(np.array(halos)[:,1:50], np.array(trees.iloc[1:].index)-np.arange(1,len(trees.index)))
split=[]
lmask=[]
for s in tqdm(spli):
    if np.log10(s[0,10])>10:
        split.append(s)
        lmask.append(len(s))
split=np.array(split, dtype=object)
# split=split

Splitting data


100%|█████████████████████████████████████████████████████| 48505/48505 [00:00<00:00, 485993.64it/s]


In [4]:
split=split[np.array(lmask)<10000]
split

array([array([[ 1.00000000e+00,  1.12296995e+09,  0.00000000e+00, ...,
                9.64280000e-01,  8.28450000e-01,  2.65367000e+00],
              [ 9.90570000e-01,  1.11337105e+09,  1.00000000e+00, ...,
                9.37690000e-01,  8.19300000e-01,  3.08823000e+00],
              [ 9.76590000e-01,  1.10357924e+09,  9.90570000e-01, ...,
                9.39730000e-01,  8.29510000e-01,  3.09167000e+00],
              ...,
              [ 8.04160000e-01,  9.45597039e+08,  8.15670000e-01, ...,
                6.59160000e-01,  4.92080000e-01,  1.80770000e-01],
              [ 7.92810000e-01,  9.35393154e+08,  8.04160000e-01, ...,
                9.34770000e-01,  6.98070000e-01,  4.14800000e-01],
              [ 7.85330000e-01,  9.25153985e+08,  7.92810000e-01, ...,
                8.76030000e-01,  7.94120000e-01, -5.23750000e-01]])   ,
       array([[1.00000000e+00, 1.12297037e+09, 0.00000000e+00, ...,
               8.68310000e-01, 8.11700000e-01, 1.49425000e+00],
              [9

In [10]:
#### Start making adjacency ####
def convert(d,p):
    dfin=[]
    if len(p)!=len(np.unique(p)):
        print('Wrong order of prog/desc')
    else:
        no=d[0]
        for desc in d:
            if desc==no:
                dfin.append(0)
            else:
                dfin.append(p.index(desc)+1)
    return dfin, np.arange(1, 1+len(p))
####
print('Creating adjacency matrix')
de, pr = [], []
for k in tqdm(range(len(split))):
    des, pro=[], []
    for i, desc in enumerate(split[k][:,3]):
        if desc in split[k][:i,1]:
            des.append(int(desc))
            pro.append(int(split[k][i,1]))
    desg, prog = convert(des,pro) #it needs to map halo 0,1,2,3 so on, not the true halo id
    de.append(desg)
    pr.append(prog)

print(f'Loading targets')
##getting the galaxy output
samp='~/../../../tigress/mcranmer/merger_trees/samout/'
ex='0_0_0/'
ex+='galprop_0-99.dat'
pdc=pd.read_table(samp+ex, skiprows=0, delimiter=',', nrows=41, header=None)
newcols=pdc.iloc[:,0]
pds=pd.read_table(samp+ex, skiprows=41, delimiter='\s+', header=None)
pds.columns=np.array(newcols)
pd0=pds[pds[pds.columns[3]]==0.00] # subhaloes
pdcen=pd0[(pd0[pds.columns[1]]==pd0[pds.columns[2]])] ##central haloes

rhalid=np.array(pdcen[pds.columns[2]])
halwgal=[]
ids=[]
out=[]
print('Matching targets and data')
for i, tree in enumerate(split):
    if tree[0,1] in rhalid:
        halwgal.append(np.hstack([tree[:,[0,2]],tree[:,9:]])) #redo class so that data columns can be chosen upon initialization
        ids.append(np.array(tree[0,1]))
        index=np.where(rhalid==tree[0,1])
        out.append(np.array(pdcen.iloc[index])[0][8]) #redo class so that target column(s) can be chosen upon initialization
halwgal=np.array(halwgal, dtype=object)
ids=np.array(ids)
out=np.log10(np.array(out))
dat=[]
for n in range(len(out)):
    edge_index = torch.tensor([pr[n],
                               de[n]], dtype=torch.long)
    x = torch.tensor(halwgal[n], dtype=torch.float)

    y=torch.tensor(out[n], dtype=torch.float)
    dat.append(Data(x=x, edge_index=edge_index, y=y, dtype=torch.float))
print('Saving dataset')
pickle.dump(dat, open(f'data_len.pkl', "wb"))

Creating adjacency matrix


100%|█████████████████████████████████████████████████████████████| 748/748 [00:17<00:00, 43.11it/s]


Loading targets
Matching targets and data
Saving dataset


In [3]:
data.root

'../../../../tigress/cj1223/graph_merge'

In [4]:
torch.cuda.is_available()
# device = torch.device('cuda')
# data = data.to(device)

True

In [5]:
from torch_geometric.loader import DataLoader

loader = DataLoader(data, batch_size=32)

In [6]:
for batch in loader:
    print(batch)

Reading data
[Batch(x=[514, 42], edge_index=[2, 513], y=[1], dtype=[1], batch=[514], ptr=[2]), Batch(x=[2193, 42], edge_index=[2, 2192], y=[1], dtype=[1], batch=[2193], ptr=[2]), Batch(x=[582, 42], edge_index=[2, 581], y=[1], dtype=[1], batch=[582], ptr=[2]), Batch(x=[619, 42], edge_index=[2, 618], y=[1], dtype=[1], batch=[619], ptr=[2]), Batch(x=[2550, 42], edge_index=[2, 2549], y=[1], dtype=[1], batch=[2550], ptr=[2]), Batch(x=[2681, 42], edge_index=[2, 2680], y=[1], dtype=[1], batch=[2681], ptr=[2]), Batch(x=[1835, 42], edge_index=[2, 1834], y=[1], dtype=[1], batch=[1835], ptr=[2]), Batch(x=[1260, 42], edge_index=[2, 1259], y=[1], dtype=[1], batch=[1260], ptr=[2]), Batch(x=[199472, 42], edge_index=[2, 199471], y=[1], dtype=[1], batch=[199472], ptr=[2]), Batch(x=[270, 42], edge_index=[2, 269], y=[1], dtype=[1], batch=[270], ptr=[2]), Batch(x=[427, 42], edge_index=[2, 426], y=[1], dtype=[1], batch=[427], ptr=[2]), Batch(x=[1395, 42], edge_index=[2, 1394], y=[1], dtype=[1], batch=[1395